In [57]:
# -*- coding: utf-8 -*-

# 必要なライブラリをすべてインポート
import cv2
import numpy as np
import os
import mediapipe as mp
import json
import glob
import pandas as pd
import datetime
import tensorflow as tf
# TF2.x形式で保存されたモデルを読み込む
from tensorflow.keras.models import load_model

class LSTM_RNN(tf.keras.Model):
    def __init__(self, n_input, n_hidden, n_classes, **kwargs):
        super(LSTM_RNN, self).__init__(**kwargs)
        self.input_dense = tf.keras.layers.Dense(n_hidden, activation='relu')
        self.time_dist_dense = tf.keras.layers.TimeDistributed(self.input_dense)
        self.lstm1 = tf.keras.layers.LSTM(n_hidden, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_hidden)
        self.out = tf.keras.layers.Dense(n_classes)

    def call(self, x):
        x = self.time_dist_dense(x)
        x = self.lstm1(x)
        x = self.lstm2(x)
        x = self.out(x)
        return x


# GPUを無効にする設定（必要に応じて）
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

## ------------------------------------------------------------------------------------
## --- 総合設定 (★★ ここをあなたの環境に合わせて必ず変更してください) ---\
## ------------------------------------------------------------------------------------\

# 1. 学習済みモデルが保存されているパス
# (例: "C:\\Users\\your_name\\weights\\my_model.h5")
MODEL_PATH = r"C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\weights\model_weights.weights.h5"

# 2. ラベルの定義 (学習時と完全に同じ順番・内容にしてください)
LABELS = [
    "ageru", "understand", "annsinnsuru", "heavy"
]

# 3. 中間ファイル・最終的な推論用ファイルを保存する親フォルダ
# このフォルダ内に 'recorded_videos', 'json_output' などのフォルダが自動生成されます。
BASE_OUTPUT_DIR = os.path.join(os.getcwd(), "Inference_Pipeline")

# 4. モデルのパラメータ設定 (学習時と完全に同じにしてください)
SEQUENCE_LENGTH = 30  # 1動画あたりのフレーム数
N_HIDDEN = 34         # LSTMの隠れ層の数

## ------------------------------------------------------------------------------------
## --- STEP 1: リアルタイム録画 (変更なし) ---\
## ------------------------------------------------------------------------------------\
def record_video(save_dir, sequence_length):
    """
    Webカメラから動画をリアルタイムで録画し、指定されたフォルダに保存する関数。
    's'キーで録画開始、'e'キーで録画停止。
    Returns:
        str: 保存された動画ファイルのフルパス。録画されなかった場合はNone。
    """
    print("--- [STEP 1] リアルタイム録画 ---")
    print("ウィンドウにフォーカスを合わせてください。")
    print("  's'キー: 録画開始")
    print("  'e'キー: 録画停止")
    print("  'q'キー: プログラム終了")
    
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("エラー: カメラを開けませんでした。")
        return None

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = 30.0

    video_writer = None
    is_recording = False
    saved_filepath = None

    # 保存ディレクトリを作成
    os.makedirs(save_dir, exist_ok=True)

    while True:
        ret, frame = cap.read()
        if not ret:
            print("エラー: フレームを読み取れませんでした。")
            break

        display_frame = frame.copy()
        
        # 録画中の表示
        if is_recording:
            cv2.circle(display_frame, (40, 40), 15, (0, 0, 255), -1)
            cv2.putText(display_frame, 'REC', (70, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
            if video_writer:
                video_writer.write(frame)

        cv2.imshow('Real-time Video Recorder', display_frame)
        key = cv2.waitKey(1) & 0xFF

        if key == ord('s') and not is_recording:
            is_recording = True
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"rec_{timestamp}.mp4"
            saved_filepath = os.path.join(save_dir, filename)
            
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            video_writer = cv2.VideoWriter(saved_filepath, fourcc, fps, (frame_width, frame_height))
            print(f"\n[録画開始] -> 保存先: {saved_filepath}")

        elif key == ord('e') and is_recording:
            print("[録画終了]")
            is_recording = False
            if video_writer:
                video_writer.release()
                video_writer = None
            break # 録画が終わったらループを抜ける

        elif key == ord('q'):
            print("プログラムを終了します。")
            if video_writer:
                video_writer.release()
            saved_filepath = None # 途中で終了したのでパスをクリア
            break
            
    cap.release()
    cv2.destroyAllWindows()
    return saved_filepath

## ------------------------------------------------------------------------------------
## --- STEP 2: MediaPipeによる骨格座標の抽出 (変更なし) ---\
## ------------------------------------------------------------------------------------\
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def extract_keypoints(results):
    keypoints = {}
    # 両手のキーポイントのみを抽出
    keypoints["left_hand"] = [{"id": i, "x": res.x, "y": res.y, "z": res.z} for i, res in enumerate(results.left_hand_landmarks.landmark)] if results.left_hand_landmarks else []
    keypoints["right_hand"] = [{"id": i, "x": res.x, "y": res.y, "z": res.z} for i, res in enumerate(results.right_hand_landmarks.landmark)] if results.right_hand_landmarks else []
    return keypoints

def extract_landmarks_from_video(video_path, json_output_dir_base, sequence_length):
    """
    単一の動画ファイルからMediaPipeで手のランドマークを抽出し、JSONファイル群として保存する。
    Returns:
        str: JSONファイルが保存されたディレクトリのパス。
    """
    print("\n--- [STEP 2] MediaPipeによる骨格座標の抽出 ---")
    if not os.path.exists(video_path):
        print(f"エラー: 動画ファイルが見つかりません: {video_path}")
        return None

    video_filename = os.path.basename(video_path)
    video_name_no_ext = os.path.splitext(video_filename)[0]
    
    # この動画用のJSON保存先フォルダを作成
    video_json_save_path = os.path.join(json_output_dir_base, video_name_no_ext)
    os.makedirs(video_json_save_path, exist_ok=True)

    cap = cv2.VideoCapture(video_path)
    print(f"処理中の動画: {video_path}")

    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        for frame_num in range(sequence_length):
            ret, frame = cap.read()
            if not ret:
                print(f"警告: 動画が{sequence_length}フレームより前に終了しました。")
                break

            _, results = mediapipe_detection(frame, holistic)
            
            keypoints = extract_keypoints(results)
            json_path = os.path.join(video_json_save_path, f'{frame_num}.json')
            with open(json_path, 'w') as f:
                json.dump(keypoints, f, indent=4)
    
    cap.release()
    cv2.destroyAllWindows()
    print(f"JSONファイルの保存が完了しました: {video_json_save_path}")
    return video_json_save_path

## ------------------------------------------------------------------------------------
## --- STEP 3: JSONから単一TXTファイルへ変換 (変更なし) ---\
## ------------------------------------------------------------------------------------\
def convert_json_to_txt(json_frames_dir, txt_output_dir):
    """
    フレームごとのJSONファイル群を、1行1フレームの単一TXTファイルに変換する。
    Returns:
        str: 生成されたTXTファイルのパス。
    """
    print("\n--- [STEP 3] JSONから単一TXTファイルへ変換 ---")
    if not os.path.isdir(json_frames_dir):
        print(f"エラー: JSONフレームのディレクトリが見つかりません: {json_frames_dir}")
        return None

    video_folder_name = os.path.basename(json_frames_dir)
    os.makedirs(txt_output_dir, exist_ok=True)
    
    # 出力ファイルパス
    output_txt_path = os.path.join(txt_output_dir, f"{video_folder_name}.txt")

    NUM_KEYPOINTS_PER_HAND = 21
    
    kps_all_frames = []
    
    # JSONファイルをフレーム順（0.json, 1.json, ...）にソートして取得
    json_files = sorted(
        glob.glob(os.path.join(json_frames_dir, "*.json")), 
        key=lambda x: int(os.path.splitext(os.path.basename(x))[0])
    )

    if not json_files:
        print(f"警告: JSONファイルが見つかりません。スキップします: {json_frames_dir}")
        return None
        
    for json_file_path in json_files:
        with open(json_file_path) as data_file:
            data = json.load(data_file)
            frame_kps_hands = []
            
            # 左手 (x, yのみ)
            left_hand = data.get("left_hand", [])
            if left_hand:
                for kp in left_hand:
                    frame_kps_hands.extend([kp.get('x', 0.0), kp.get('y', 0.0)])
            else:
                frame_kps_hands.extend([0.0] * (NUM_KEYPOINTS_PER_HAND * 2))

            # 右手 (x, yのみ)
            right_hand = data.get("right_hand", [])
            if right_hand:
                for kp in right_hand:
                    frame_kps_hands.extend([kp.get('x', 0.0), kp.get('y', 0.0)])
            else:
                frame_kps_hands.extend([0.0] * (NUM_KEYPOINTS_PER_HAND * 2))
                
            kps_all_frames.append(frame_kps_hands)

    # テキストファイルに書き込み
    with open(output_txt_path, "w") as text_file:
        for frame_data in kps_all_frames:
            line = ",".join(map(str, frame_data))
            text_file.write(line + '\n')
            
    print(f"TXTファイルへの変換が完了しました: {output_txt_path}")
    return output_txt_path

## ------------------------------------------------------------------------------------
## --- STEP 4: 推論用データ(X_val.txt)の作成 (変更なし) ---\
## ------------------------------------------------------------------------------------\
def prepare_data_for_lstm(source_txt_path, final_output_path, num_steps, overlap=0.8125):
    """
    単一のTXTファイルを読み込み、線形補間とスライディングウィンドウを適用して
    LSTM推論用の最終的な入力ファイル(X_val.txt)を作成する。
    Returns:
        bool: 成功した場合はTrue、失敗した場合はFalse。
    """
    print("\n--- [STEP 4] 推論用データ(X_val.txt)の作成 ---")
    if not os.path.exists(source_txt_path):
        print(f"エラー: ソースTXTファイルが見つかりません: {source_txt_path}")
        return False
    
    try:
        # データを読み込み、0.0をNaNとして扱う
        data_df = pd.read_csv(source_txt_path, header=None, na_values=0.0)
        
        # 線形補間
        data_df.interpolate(method='linear', axis=0, inplace=True, limit_direction='both')
        # それでも残っているNaNを0.0で埋める
        data_df.fillna(0.0, inplace=True)
        
        num_frames = len(data_df)
        if num_frames < num_steps:
            print(f"エラー: フレーム数が不足しています({num_frames}フレーム)。最低{num_steps}フレーム必要です。")
            return False

        # スライディングウィンドウでシーケンスを生成
        step_size = int(num_steps * (1 - overlap))
        if step_size < 1: step_size = 1 # ゼロ除算を避ける
        
        sequences = []
        for start_frame in range(0, num_frames - num_steps + 1, step_size):
            end_frame = start_frame + num_steps
            sequence = data_df.iloc[start_frame:end_frame].values
            sequences.append(sequence) # flattenしない
        
        if not sequences:
            print("エラー: シーケンスを生成できませんでした。")
            return False

        # 最終的なデータをファイルに書き込み
        with open(final_output_path, 'w') as f:
            for seq in sequences:
                np.savetxt(f, seq, delimiter=',')
        
        print(f"{len(sequences)}個のシーケンスを生成し、推論用ファイルに保存しました: {final_output_path}")
        return True

    except Exception as e:
        print(f"データ準備中にエラーが発生しました: {e}")
        return False




## ------------------------------------------------------------------------------------
## --- STEP 5: LSTMモデルによる推論 (★★ TensorFlow 2.x に更新) ---\
## ------------------------------------------------------------------------------------\

    

# def run_inference(x_val_path, model_path, labels, n_steps, n_hidden):
#     """
#     準備された推論用データと学習済みモデルを使用して、手話の推論を実行する。
#     """
#     print("\n--- [STEP 5] LSTMモデルによる推論 ---")
    
#     n_classes = len(labels)
    
#     # -- データ読み込みヘルパー関数 --
#     def load_X(X_path):
#         try:
#             X_ = np.loadtxt(X_path, delimiter=',', dtype=np.float32)
#             blocks = int(len(X_) / n_steps)
#             X_ = np.array(np.split(X_, blocks))
#             return X_
#         except FileNotFoundError:
#             print(f"エラー: 推論用ファイルが見つかりません - {X_path}")
#             return None
#         except Exception as e:
#             print(f"データ読み込み中にエラーが発生しました: {e}")
#             return None
            
#     # -- メイン処理 --
#     X_val = load_X(x_val_path)
#     if X_val is None or len(X_val) == 0:
#         print("推論データが空のため、処理を中止します。")
#         return

#     n_input = X_val.shape[2]
    
#     # -- モデルの読み込み --
#     try:
        

#         # カスタムクラスを登録して読み込む
#         model = load_model(
#             MODEL_PATH,
#             custom_objects={'LSTM_RNN': LSTM_RNN}
#         )           

#         print(f"モデルが期待する入力形状: {model.input_shape}")
#         print(f"モデルを正常に読み込みました: {model_path}")
#         model.summary() # モデルの構造を表示
#         print(f"現在のスクリプトが生成したデータの形状: {X_val.shape}")
        
#     except Exception as e:
#         print(f"モデルの読み込み中にエラーが発生しました: {e}")
#         print("MODEL_PATHが正しいか、TensorFlow 2.x形式で保存されたモデルか確認してください。")
#         print("もしモデルがない場合は、先にTF2.xでモデルを学習・保存する必要があります。")
#         return

#     # -- 推論の実行 --
#     try:
#         # model.predict()で一括して予測
#         predictions_probabilities = model.predict(X_val)
#         # 最も確率の高いクラスのインデックスを取得
#         predictions = np.argmax(predictions_probabilities, axis=1)

#         print("\n--- [最終推論結果] ---")
        
#         # 各シーケンスの予測結果から、最も多く予測されたラベルを最終結果とする
#         if len(predictions) > 0:
#             counts = np.bincount(predictions, minlength=n_classes)
#             final_prediction_index = np.argmax(counts)
#             final_prediction_label = labels[final_prediction_index]
#             print(f"👉 録画された手話の動作は『 {final_prediction_label} 』です。")
#         else:
#             print("予測可能なデータがありませんでした。")

#     except Exception as e:
#         print(f"推論中にエラーが発生しました: {e}")
def run_inference(x_val_path, model_path, labels, n_steps, n_hidden):
    """
    x_val_path: 'final_input/X_val.txt' のフルパス（関数呼び出し側で渡す）
    model_path: .weights.h5 のフルパス
    labels: ラベルリスト（学習時と同じ順序）
    n_steps: シーケンス長（例: 30）
    n_hidden: LSTMのユニット数（学習時と同じ）
    """
    print("\n--- [STEP 5] LSTMモデルによる推論 ---")

    try:
        # -------------------------
        # 1) 推論用データ読み込み
        # -------------------------
        if not os.path.exists(x_val_path):
            print(f"エラー: 推論用ファイルが見つかりません: {x_val_path}")
            return

        X_loaded = np.loadtxt(x_val_path, delimiter=',', dtype=np.float32)
        if X_loaded.size == 0:
            print("エラー: 読み込んだデータが空です。")
            return

        # X_loaded は (rows, features) になるはず
        if X_loaded.ndim == 1:
            # 1行しか無い場合 -> (1, features)
            X_loaded = X_loaded.reshape(1, -1)

        rows, features = X_loaded.shape
        if rows % n_steps != 0:
            # 多くの場合、stepsの倍数になっているはず
            print(f"警告: 読み込んだ行数({rows})がシーケンス長({n_steps})の倍数ではありません。")
            # 可能なら切り捨てして進める（安全策）
            valid_rows = (rows // n_steps) * n_steps
            if valid_rows == 0:
                print("エラー: 有効なシーケンスがありません。処理を中止します。")
                return
            X_loaded = X_loaded[:valid_rows, :]
            rows = valid_rows
            print(f"→ {valid_rows} 行に切り捨てました。")
        
        blocks = rows // n_steps
        X_val = X_loaded.reshape(blocks, n_steps, features)
        print(f"読み込んだデータ: rows={rows}, features={features}, blocks={blocks}, shape={X_val.shape}")

        # n_input を自動決定（1フレームあたりの特徴量）
        n_input = features
        n_classes = len(labels)

        # -------------------------
        # 2) モデル構築（構造を学習時と合わせる）
        # -------------------------
        model = LSTM_RNN(n_input, n_hidden, n_classes)

        # 確実に内部のレイヤーが構築されるようにダミー入力を一度通す
        dummy = tf.zeros((1, n_steps, n_input), dtype=tf.float32)
        _ = model(dummy)  # これで内部がビルドされる

        # モデルのサマリ（unbuiltが出る場合はここで確認）
        model.summary()

        # -------------------------
        # 3) 重み読み込み
        # -------------------------
        if not os.path.exists(model_path):
            print(f"エラー: モデル重みファイルが見つかりません: {model_path}")
            return

        model.load_weights(model_path)
        print(f"モデル重みを読み込みました: {model_path}")

        # -------------------------
        # 4) 推論（バッチで）
        # -------------------------
        preds = model.predict(X_val)  # shape: (blocks, n_classes)
        #print(f"preds shape: {preds.shape}")

        # 各シーケンスごとの予測クラス
        pred_indices = np.argmax(preds, axis=1)
        #print(f"各シーケンスの予測インデックス: {pred_indices}")

        # 全シーケンスの多数決で最終ラベルを決める（元のロジックと同じ）
        counts = np.bincount(pred_indices, minlength=n_classes)
        final_idx = np.argmax(counts)
        final_label = labels[final_idx]

        print("\n--- [最終推論結果] ---")
        print(f"録画された手話の動作は『 {final_label} 』です")
        #print(f"各クラスのカウント: {counts}")
        #print(f"各シーケンス確率サンプル(最初の2件):\n{preds[:min(2, len(preds))]}")

    except Exception as e:
        print(f"モデルの読み込みまたは推論中にエラーが発生しました: {e}")
        import traceback
        traceback.print_exc()





## ------------------------------------------------------------------------------------
## --- パイプライン実行メイン処理 (変更なし) ---\
## ------------------------------------------------------------------------------------\
if __name__ == '__main__':
    # 各ステップで使用するディレクトリを準備
    video_save_dir = os.path.join(BASE_OUTPUT_DIR, "recorded_videos")
    json_output_dir = os.path.join(BASE_OUTPUT_DIR, "json_output")
    txt_output_dir = os.path.join(BASE_OUTPUT_DIR, "txt_converted")
    final_input_dir = os.path.join(BASE_OUTPUT_DIR, "final_input")
    final_x_val_path = os.path.join(final_input_dir, "X_val.txt")

    for d in [video_save_dir, json_output_dir, txt_output_dir, final_input_dir]:
        os.makedirs(d, exist_ok=True)
    
    # STEP 1: リアルタイム録画
    recorded_video_path = record_video(video_save_dir, SEQUENCE_LENGTH)
    
    # パイプライン処理
    if recorded_video_path:
        try:
            # STEP 2: MediaPipeによる骨格座標の抽出
            json_frames_dir = extract_landmarks_from_video(recorded_video_path, json_output_dir, SEQUENCE_LENGTH)
            if json_frames_dir is None: raise Exception("骨格座標の抽出に失敗しました。")

            # STEP 3: JSONから単一TXTファイルへ変換
            converted_txt_path = convert_json_to_txt(json_frames_dir, txt_output_dir)
            if converted_txt_path is None: raise Exception("TXTファイルへの変換に失敗しました。")
            
            # STEP 4: 推論用データ(X_val.txt)の作成
            success = prepare_data_for_lstm(converted_txt_path, final_x_val_path, SEQUENCE_LENGTH)
            if not success: raise Exception("推論用データの作成に失敗しました。")

            # STEP 5: LSTMモデルによる推論
            run_inference(final_x_val_path, MODEL_PATH, LABELS, SEQUENCE_LENGTH, N_HIDDEN)

        except Exception as e:
            print(f"\nパイプラインの処理中にエラーが発生しました: {e}")
    else:
        print("\n録画がキャンセルされたため、処理を終了しました。")

--- [STEP 1] リアルタイム録画 ---
ウィンドウにフォーカスを合わせてください。
  's'キー: 録画開始
  'e'キー: 録画停止
  'q'キー: プログラム終了

[録画開始] -> 保存先: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\Inference_Pipeline\recorded_videos\rec_20251012_155501.mp4
[録画終了]

--- [STEP 2] MediaPipeによる骨格座標の抽出 ---
処理中の動画: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\Inference_Pipeline\recorded_videos\rec_20251012_155501.mp4
JSONファイルの保存が完了しました: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\Inference_Pipeline\json_output\rec_20251012_155501

--- [STEP 3] JSONから単一TXTファイルへ変換 ---
TXTファイルへの変換が完了しました: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\Inference_Pipeline\txt_converted\rec_20251012_155501.txt

--- [STEP 4] 推論用データ(X_val.txt)の作成 ---
1個のシーケンスを生成し、推論用ファイルに保存しました: c:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\Inference_Pipeline\final_input\X

Model: "lstm_rnn_53"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_106 (Dense)               │ (1, 34)                │         2,890 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_53             │ (1, 30, 34)            │         2,890 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_106 (LSTM)                 │ (1, 30, 34)            │         9,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_107 (LSTM)                 │ (1, 34)                │         9,384 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_107 (Dense)               │ (1, 4)                 │           140 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21,798 (85.15 KB)

 Trainable params: 21,798 (85.15 KB)

 Non-trainable params: 0 (0.00 B)

モデル重みを読み込みました: C:\Users\admin\Downloads\RNN-for-Human-Activity-Recognition-using-2D-Pose-Input-master\weights\model_weights.weights.h5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 353ms/step

--- [最終推論結果] ---
録画された手話の動作は『 understand 』です
